## Converting the riding boundary shapefile into geojson, topojson 

* TopoJSON Wiki: https://github.com/mbostock/topojson/wiki
* Dorling Cartograms: http://www.ncgia.ucsb.edu/projects/Cartogram_Central/types.html
* TopoJSON API doc: https://github.com/mbostock/topojson/wiki/API-Reference

In [1]:
import subprocess

In [2]:
def exec_command(command_text):
    p = subprocess.call(command_text, shell=True)
    if p == 1: raise Exception('failed command') 

In [3]:
data_path = "../data/fed_cf_CA_2_1_shp_en/"

These are the steps from the Let's make a Map tutorial: http://bost.ocks.org/mike/map/

In [10]:
exec_command("ogr2ogr -f GeoJSON " + data_path + "places.json " + data_path + "FED_CA_2_1_en.shp")

In [11]:
exec_command("topojson -o " + data_path + "ca.json -- " + data_path + "places.json")

That only renders as a black rectangle, and it takes a really long time.

Loading the shapefile into google earth shows the ridings fine, so I don't think the shapefile is corrupted. 

Let's try simplifying the geometry.

In [14]:
exec_command("ogr2ogr -f GeoJSON " + data_path + "ridings.json " + data_path + "FED_CA_2_1_en.kml")

Exception: failed command

More documentation: http://www.gdal.org/drv_shapefile.html